In [ ]:
import functools
import numpy
import random

epsilon = 1e-10

In [1]:
class Candidate:
    def __init__(self):
        self.crowding_distance = 0
        self.domination_count = 0
        self.rank = 0
        self.objective = 0
        self.dominated_candidates = None
        self.features = None
        self.dominates = None


In [ ]:
class Problem:
    def __init__(self, objectives, bounds, dims, fitness):
        self.objectives = objectives
        self.bounds = bounds
        self.dims = dims
        self.n = len(bounds)
        self.fitness = fitness
        
    def __dominates(self, candidateA, candidateB):
        fA = self.fitness(candidateA.features)
        fB = self.fitness(candidateB.features)
        
        not_dominated = all(map(lambda f: f[0] <= f[1], zip(fA, fB)))
        dominates = any(map(lambda f: f[0] < f[1], zip(fA, fB)))
        
        return dominates and not_dominated
    
    def _calculate_objectives(self, candidate):
        objectives = []
        for objective in self.fitness(candidate.features):
            objectives.append(objective)
            
        return objectives
        
    def generate_candidate(self):
        candidate = Candidate()
        candidate.features = []
        candidate.dominates = functools.partial(self.__dominates,
                                                candidateA=candidate)
        for idx in range(self.n):
            candidate.features += random.uniform(self.bounds[idx][0], 
                                                 self.bounds[idx][1])
        candidate.objective = self._calculate_objectives(candidate)
            
        return candidate


In [8]:
class NSGA2:
    def __init__(self, problem, pop_size, n_tpart, max_gen, cr, m, seed=42.0):
        self.pop_size = pop_size        # Population size
        self.max_gen = max_gen          # Max generations to evolve for
        self.n_tpart = n_tpart          # Number of tournament participants
        self.cr = cr                    # Cross-over probability
        self.m = m                      # Mutation probability
        self.eps = 1e-3                 # Mutation strength
        self.seed = seed                # Random seed
        
        self.problem = problem          # Instance of the problem, ie. objective function/s
        
        self.population = None
        
        self.initialize()
        
    def initialize(self):
        self.population = []
        random.seed(self.seed)
        
        for idx in range(self.pop_size):
            self.population += self.problem.generate_candidate()
    
    def _mutate(self, candidate, n=1):
        for idx in range(len(candidate.features)):
            if random.random() > self.m:
                candidate.features[idx] += self.eps * (random.random() - 1) / 2
                if candidate.features[idx] > self.problem.bounds[idx][1]:
                    candidate.features[idx] = self.problem.bounds[idx][1]
                elif candidate.features[idx] < self.problem.bounds[idx][0]:
                    candidate.features[idx] = self.problem.bounds[idx][0]
                    
        return candidate
    
    def _crossover(self, candidateA, candidateB, n=1):
        for idx in range(len(candidateA.features)):
            if random.random() > self.cr:
                candidateA.features[idx], candidateB.features[idx]\
                = candidateB.features[idx], candidateA.features[idx]
                
        return candidateA, candidateB
    
    def _tournament(self):
        participants = random.sample(self.population, self.n_tpart)
        
        return max(participants,
                   key=lambda x: x.crowding_distance)
    
    def _create_offsprings(self):
        offsprings = []
        
        while len(offsprings) < self.pop_size:
            parentA = self._tournament()
            parentB = parentA

            while parentA.features == parentB.features:
                parentB = self._tournament()

            childA, childB == self._crossover(parentA, parentB)
            childA = self._mutate(childA)
            childA.objectives = self.problem._calculate_objectives(childA)
            
            childB = self._mutate(childB)
            childB.objectives = self.problem._calculate_objectives(childB)
            
            offsprings += [childA, childB]
            
        return offsprings
        
    def _non_dominated_sort(self):
        fronts = [[]]
        
        for candidateA in self.population:
            candidateA.domination_count = 0
            candidateA.dominated_candidates = set()
            
            for candidateB in self.population:
                if candidateA.dominates(candidateB):
                    candidateA.dominated_candidates.add(candidateB)
                elif candidateB.dominates(candidateA):
                    candidateA.domination_count += 1
                    
            if candidateA.domination_count == 0:
                candidateA.rank = 0
                fronts[0].append(candidateA)
        
        idx = 0
        while len(fronts[idx]) > 0:
            ith_front = []
            
            for candidateA in fronts[idx]:
                for candidateB in candidateA.dominated_candidates:
                    candidateB.domination_count -= 1
                    
                    if candidateB.domination_count == 0:
                        candidateB.rank = i + 1
                        ith_front.append(candidateB)
                        
            i += 1
            fronts.append(ith_front)
            
        return fronts           
    
    def _calculate_crowding_distance(self, front):
        if len(front) > 0:
            for candidate in front:
                candidate.crowding_distance = 0
            
            for m in range(len(self.problem.objectives)):
                front = sorted(front,
                               key=lambda x: x.objectives[m])
                front[0].crowding_distance = float('inf')
                front[-1].crowding_distance = float('inf')
                
                for idx in range(1, len(front)-2): 
                    front[idx].crowding_distance +=\
                        (front[idx+1].objectives[m] - front[idx+1].objectives[m])/\
                                (self.problem.bounds[1] - self.problem.bounds[0])               
                
    def _crowding_operator(self, candidateA, candidateB):
        return candidateA.rank < candidateB.rank or\
                (candidateA.rank == candidateB.rank and\
                 candidateA.crowding_distance > candidateB.crowding_distance)
        
    def solve(self):
        pass
        

In [3]:
def f1(x):
    return x ** 2

def f2(x):
    return (x - 2) ** 2

def fitness(candidate):
    """minimise for both f1, f2 - so max fitness at optima"""
    return 1/(f1(candidate[0]) + epsilon), 1/(f2(candidate[1]) + epsilon)

objectives = [f1, f2]  